In [27]:
import sqlite3 as sql
import pandas as pd
import glob as glob
import os
import zipfile

In [65]:
# dest_folder = '\\Databases'

In [61]:
db_name = "dvdrental.db"

In [62]:
origin = "csv_zip.zip"

In [63]:
#with sql.connect(os.path.join(dest_folder, db_name)) as conn:
with sql.connect(db_name) as conn:
    # df_dict = {}
    with zipfile.ZipFile(origin) as z:
        for file_name in z.namelist():
            if file_name.endswith('.csv'):
                table = os.path.splitext(file_name)[0]
                with z.open(file_name) as f:
                    df = pd.read_csv(f)
                    df.to_sql(table, conn, if_exists='replace', index=False)
                    # df_dict[os.path.splitext(file_name)[0]] = pd.read_csv(f)
                    print(f'Loaded {file_name} into table {table}')

Loaded country.csv into table country
Loaded film_category.csv into table film_category
Loaded store.csv into table store
Loaded staff.csv into table staff
Loaded film.csv into table film
Loaded city.csv into table city
Loaded address.csv into table address
Loaded customer.csv into table customer
Loaded inventory.csv into table inventory
Loaded category.csv into table category
Loaded actor.csv into table actor
Loaded language.csv into table language
Loaded film_actor.csv into table film_actor
Loaded rental.csv into table rental


In [66]:
# db = os.path.join(dest_folder, db_name)

In [67]:
with sql.connect(db_name) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
    tables = cursor.fetchall()

    print("Tables in db:")

    for table in tables:
        print("-", table[0])

Tables in db:
- country
- film_category
- store
- staff
- film
- city
- address
- customer
- inventory
- category
- actor
- language
- film_actor
- rental


In [ ]:
def query_func(dbase, query):
    with sql.connect(dbase) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    